# Inquiry into Kids First to FHIR model mapping

## Build a ship of pyrates

In [1]:
from fhir_pyrate import Pirate, Ahoy
import pandas as pd

pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)

PROD_URL = 'https://kf-api-fhir-service.kidsfirstdrc.org/'
DEV_URL = 'https://kf-api-fhir-service-dev.kidsfirstdrc.org/'
QA_URL = 'https://kf-api-fhir-service-qa.kidsfirstdrc.org'

ENDPOINT_URLS = {
    'PROD_URL' : 'https://kf-api-fhir-service.kidsfirstdrc.org/',
    'QA_URL' : 'https://kf-api-fhir-service-qa.kidsfirstdrc.org',
    'DEV_URL' : 'https://kf-api-fhir-service-dev.kidsfirstdrc.org/'
}

def get_pirate(url):
    auth = Ahoy(
        username="wnkhan32@gmail.com",
        auth_method=None,
        auth_url=url
    )

    pirate = Pirate(
        auth=auth, 
        base_url=url, 
        print_request_url=False, 
        num_processes=1
    )   

    return pirate

ship = {
    url_label : get_pirate(url) 
    for url_label, url in ENDPOINT_URLS.items()
}

## Check Pyrate Booty

In [2]:
import requests

def get_studies(pirate: Pirate):
    studies = []
    try:
        studies_df = pirate.steal_bundles_to_dataframe(
            resource_type='ResearchStudy',
            fhir_paths=[
                ('study_ids','meta.tag[0].code'),
            ]
        )
        if isinstance(studies_df,pd.DataFrame) and 'study_ids' in studies_df.columns:
            studies = studies_df['study_ids'].to_list()
    except (requests.exceptions.HTTPError) as e:
        print(e.response)

    return studies

endpoint_studies = {
    label : get_studies(pirate)
    for label, pirate in ship.items()
}

endpoint_studies

Query & Build DF (ResearchStudy): 100%|██████████| 1/1 [00:00<00:00, 1884.23it/s]

Failed to get data from: https://kf-api-fhir-service-qa.kidsfirstdrc.org/ResearchStudy?
Failed to get data from: https://kf-api-fhir-service-dev.kidsfirstdrc.org/ResearchStudy?


{'PROD_URL': ['SD_65064P2Z', 'SD_Y6VRG6MD', 'SD_FYCR78W0', 'SD_T8VSYRSG'],
 'QA_URL': [],
 'DEV_URL': []}

## Check Stuides in Pyrate Booty

In [3]:
metadata = ship['PROD_URL'].steal_bundles_to_dataframe(
    resource_type='ResearchStudy',
    fhir_paths=[
        ('study_ids','meta.tag[0].code'),
        ('study_type','keyword[1].coding[0].code')
    ]
)

metadata


Query & Build DF (ResearchStudy): 100%|██████████| 1/1 [00:00<00:00, 1309.49it/s]


,study_ids,study_type
0,SD_65064P2Z,INCLUDE
1,SD_Y6VRG6MD,INCLUDE
2,SD_FYCR78W0,INCLUDE
3,SD_T8VSYRSG,INCLUDE


## Noteworthy results
1. Creation time not identified
2. No column identified for age_at_enrollment

# KF FHIR Patient &rarr; C2M2 Subject
| C2M2 Field | FHIR Field |
| ---------- | ---------- |
| local_id   | Patient.identifier[].value |
| project_local_id | Patient.meta_tag[].code |
| sex | Patient.gender |
| ethnicity | Patient.extension_1_extension_0_valueString |
 


In [4]:
patient_df = ship['PROD_URL'].steal_bundles_to_dataframe(
    resource_type="Patient",
    fhir_paths=[
        ("kf_id","identifier[0].value"),
        ("study_id","meta.tag[0].code"),
        ("sex","gender"),
        ("ethnicity","extension[1].extension[0].valueString"),
    ]
)

patient_df.head()

Query & Build DF (Patient): 40it [00:04,  9.92it/s]


,kf_id,study_id,sex,ethnicity
0,PT_TMQWP2F8,SD_65064P2Z,female,Not Reported
1,PT_ESANSF5N,SD_65064P2Z,female,Not Hispanic or Latino
2,PT_QV8RJTJY,SD_65064P2Z,female,Not Hispanic or Latino
3,PT_FV1C37JJ,SD_65064P2Z,male,Not Hispanic or Latino
4,PT_F2YJRHS0,SD_65064P2Z,female,Not Hispanic or Latino
